# TAP_verify_DP0.1-general

<br>Owner: **Douglas Tucker** ([@douglasleetucker](https://github.com/LSSTScienceCollaborations/StackClub/issues/new?body=@douglasleetucker))
<br>Updated for DC2 by:   Douglas Tucker  following in part work for DESC by Yao-Yuan Mao (@yymao) and Johann Cohen-Tanugi (@johannct)
<br>Last Verified to Run: **2021-08-17**
<br>Verified Stack Release: **w_2021_25**

### Objectives

This notebook is meant to run tests of the basic content of DP0.1 TAPserver tables in the `dp01_dc2_catalogs` schema on the IDF.

### Logistics

This notebook is intended to be runnable on `data.lsst.cloud` from a local git clone of https://github.com/LSSTScienceCollaborations/StackClub.


In [1]:
# https://jira.lsstcorp.org/browse/PREOPS-473
# 
# create a separate notebook "for general consumption" (with more notes).
# More important now than comparing against parquet files.
# "Here's a description of what the DB looks like. -- a good product to hand off."
# Maybe do some of this within the SQL query:  can SQL test for NaN's?"
# Point towards the data, not so much the DB mechanics.
# Cols:  max, min, 1%, 99%
# qserv has a data return limit of 5GB; so beware!

### Set Up

In [2]:
# Import general python packages
import numpy as np
import pandas as pd
import os
from datetime import datetime
import gc

In [3]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [4]:
# Import the Rubin TAP service utilities
from rubin_jupyter_utils.lab.notebook import get_tap_service, retrieve_query, get_catalog

# Get an instance of the TAP service
service = get_tap_service()
assert service is not None
assert service.baseurl == "https://data.lsst.cloud/api/tap"

Patching auth into notebook.base.handlers.IPythonHandler(notebook.base.handlers.AuthenticatedHandler) -> IPythonHandler(jupyterhub.singleuser.mixins.HubAuthenticatedHandler, notebook.base.handlers.AuthenticatedHandler)


### User Input

In [11]:
# Which table are we interested in for this notebook?
schema_name = 'dp01_dc2_catalogs'

# Output directory and output file basename info
outputDir = '/home/douglasleetucker/WORK'
baseName = 'TAP_verify_DP01'

# Debug option for quick results for testing purposes:
#    True:  only runs over a few tracts of data
#    False:  runs over all tracts available
debug = True

# Do final cleanup?
do_final_cleanup = False

# Level of output to screen (0, 1, 2, ...)
verbose = 2

### Basic Tests

Since some of the following queries take time, let's measure how long it takes to complete each query cell.  We will use `%%time` to estimate processor and wall clock time, and `datetime.now()` to estimate the total time to send the query and download its results.

In [58]:
%%time
now0=datetime.now()

query = """SELECT table_name FROM TAP_SCHEMA.tables WHERE schema_name=%s""" % ("\'"+schema_name+"\'")
print(query)
results = service.search(query)
df = results.to_table().to_pandas()
table_full_name_list = df['table_name'].tolist()
print(table_full_name_list)

SELECT table_name FROM TAP_SCHEMA.tables WHERE schema_name='dp01_dc2_catalogs'
['dp01_dc2_catalogs.forced_photometry', 'dp01_dc2_catalogs.object', 'dp01_dc2_catalogs.position', 'dp01_dc2_catalogs.reference', 'dp01_dc2_catalogs.truth_match']
CPU times: user 17 ms, sys: 0 ns, total: 17 ms
Wall time: 60.7 ms


In [59]:
%%time
now0=datetime.now()

for table_full_name in table_full_name_list:
    
    query = """SELECT COUNT(*) FROM %s""" % (table_full_name)
    results = service.search(query)
    df = results.to_table().to_pandas()
    ntot = df['COUNT'].loc[0]
    outputLine = """%40s:  %d""" % (table_full_name, ntot)
    print(outputLine)

now1=datetime.now()
print("Total time:", now1-now0)

     dp01_dc2_catalogs.forced_photometry:  147088445
                dp01_dc2_catalogs.object:  147088478
              dp01_dc2_catalogs.position:  147088445
             dp01_dc2_catalogs.reference:  147088445
           dp01_dc2_catalogs.truth_match:  765823615
Total time: 0:05:36.804264
CPU times: user 58.8 ms, sys: 4.07 ms, total: 62.8 ms
Wall time: 5min 36s
